<a href="https://colab.research.google.com/github/dinuka-rp/L6-AI/blob/main/Prasan_Yapa/CNN-Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Recognition with CNN using TensorFlow and Keras

 TensorFlow is an open source library created for Python by the Google Brain team. TensorFlow compiles many different algorithms and models together, enabling the user to implement deep neural networks for use in tasks like image recognition/classification and natural language processing.

Keras is a high-level API that can use TensorFlow's
functions underneath.

Image recognition refers to the task of inputting an image into a neural network and having it output label for that image. The label that the network outputs will correspond to a pre-defined class.

Features are the elements of the data that you care about which will be fed through the network. In the specific case of image recognition, the features are the groups of pixels, like edges and points, of an object that the network will analyze for patterns.


CIFAR-10 dataset. CIFAR-10 is a large image dataset containing over 60,000 images representing 10 different classes of objects like cats, planes, and cars. 

One great thing about the CIFAR-10 dataset is that it comes prepackaged with Keras, so it is very easy to load up the dataset and the images need very little preprocessing.


## Loading dataset

In [14]:
import numpy
from keras.datasets import cifar10



We're going to be using a random seed here so that the results achieved in this lab can be replicated by you, which is why we need numpy.


In [15]:
seed = 21
numpy.random.seed(seed)
# https://stackoverflow.com/questions/21494489/what-does-numpy-random-seed0-do

Now let's load in the dataset. We can do so simply by specifying which variables we want to load the data into, and then using the load_data() function.


In [16]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In most cases you will need to do some preprocessing of your data to get it ready for use, but since we are using a prepackaged dataset, very little preprocessing needs to be done. One thing we want to do is normalize the input data.


So, in order to normalize the data, we can simply divide the image values by 255. To do this we first need to make the data a float type, since they are currently integers. We can do this by using the `astype()` Numpy command and then declaring what data type we want.


In [17]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

We are effectively doing binary classification here because an image either belongs to one class or it doesn't, it can't fall somewhere in-between. The Numpy command `to_categorical()` is used to one-hot encode. This is why we imported the `np_utils` function from Keras, as it contains `to_categorical().`


In [18]:
from keras.utils import np_utils

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

# print(y_test)
print(num_classes)

10


## Design the model

The first thing to do is define the format we would like to use for the model, Keras has several different formats or blueprints to build models on, but Sequential is the most commonly used, and for that reason, we have imported it from Keras.


In [19]:
from keras.models import Sequential

model = Sequential()

When implementing these in Keras, we must specify the number of channels/filters we want (that's the 32 below), the size of the filter we want (3 x 3 in this case), the input shape (when creating the first layer) and the activation and padding we need. As mentioned, relu is the most common activation, and padding='same' just means we aren't changing the size of the image at all.


In [20]:
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D

model.add(Conv2D(32, (3, 3), input_shape=X_train.shape[1:], padding='same'))
model.add(Activation('relu'))


We will make a dropout layer to prevent overfitting, which functions by *randomly eliminating some of the connections between the layers*.


In [21]:
model.add(Dropout(0.2))

We may also want to do batch normalization here. Batch Normalization normalizes the inputs heading into the next layer, ensuring that the network always creates activations with the same distribution that we desire.


In [22]:
model.add(BatchNormalization())

Now comes another convolutional layer, but the filter size increases so the network can learn more complex representations.

In [23]:
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))


Here's the pooling layer, as discussed before this helps make the image classifier more robust so it can learn relevant patterns. There's also the dropout and batch normalization.


In [24]:
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

That's the basic flow for the first half of a CNN implementation:


1.   Convolutional
2.   activation
3.   dropout
4.   pooling

It's important not to have too many pooling layers, as *each pooling discards some data*.
Pooling too often will lead to there being almost nothing for the densely connected layers to learn about when the data reaches them. 

You can now repeat these layers to give your network more representations to work off.


In [25]:
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

After we are done with the convolutional layers, we need to Flatten the data, which is why we imported the function above.
We'll also add a layer of dropout again.


In [26]:
model.add(Flatten())
model.add(Dropout(0.2))

Now we make use of the Dense import and create the first densely connected layer. We need to specify the number of neurons in the dense layer.

Note that the numbers of neurons in succeeding layers decreases, *eventually approaching the same number of neurons as there are classes in the dataset*.


In [27]:
from keras.constraints import maxnorm

model.add(Dense(256, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(128, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

Finally, the ***softmax activation function*** *selects the neuron with the highest probability as its output, voting that the image belongs to that class*.

In [28]:
model.add(Dense(num_classes))
model.add(Activation('softmax'))

## Train the model

Now that we've designed the model we want to use, we just have to compile it. Let's specify the number of epochs we want to train for, as well as the optimizer we want to use.

The optimizer is what will tune the weights in your network to approach the point of lowest loss.
The *Adam* algorithm is one of the most commonly used optimizers because it gives great performance on most problems.


In [29]:
epochs = 25
optimizer = 'Adam'

Let's now compile the model with our chosen parameters. Let's also specify a metric to use.


In [30]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0

Now we get to training the model. To do this, all we have to do is call the `fit()` function on the model and pass in the chosen parameters.


In [31]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs,
batch_size=64)

Epoch 1/25
782/782 [==============================] - 402s 512ms/step - loss: 1.5098 - accuracy: 0.4655 - val_loss: 1.1239 - val_accuracy: 0.5873
Epoch 2/25
782/782 [==============================] - 400s 512ms/step - loss: 1.0231 - accuracy: 0.6390 - val_loss: 0.8155 - val_accuracy: 0.7144
Epoch 3/25
782/782 [==============================] - 402s 514ms/step - loss: 0.8524 - accuracy: 0.6993 - val_loss: 0.7520 - val_accuracy: 0.7378
Epoch 4/25
782/782 [==============================] - 404s 517ms/step - loss: 0.7625 - accuracy: 0.7335 - val_loss: 0.6692 - val_accuracy: 0.7692
Epoch 5/25
782/782 [==============================] - 404s 517ms/step - loss: 0.7100 - accuracy: 0.7501 - val_loss: 0.6568 - val_accuracy: 0.7696
Epoch 6/25
782/782 [==============================] - 403s 516ms/step - loss: 0.6596 - accuracy: 0.7685 - val_loss: 0.6555 - val_accuracy: 0.7729
Epoch 7/25
782/782 [==============================] - 403s 515ms/step - loss: 0.6268 - accuracy: 0.7803 - val_loss: 0.6580 -

Finally, we can evaluate the model and see how it performed. Just call `model.evaluate()`.


In [32]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 81.76%
